In [2]:
import numpy
import pandas as pd
import os
import sys
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pymysql

In [5]:
host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
SQL_Query = pd.read_sql_query(
'''select * from tweets_with_location ''', con)

In [6]:
df = pd.DataFrame(SQL_Query)

In [7]:
spark = SparkSession \
    .builder \
    .appName("data_import") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.shuffle.service.enabled", "true") \
    .enableHiveSupport() \
    .getOrCreate()

In [8]:
df_sp = spark.createDataFrame(df)

In [9]:
df_sp.show()

+--------+---------------+-------------------+--------------------+-------------+-----------+-----+
|tweet_id|       username|         created_at|               tweet|retweet_count|   location|place|
+--------+---------------+-------------------+--------------------+-------------+-----------+-----+
|       1|    Not_Silvers|2021-01-11 21:32:20|RT @tamrieldude: ...|            0|       Ohio| null|
|       2|  EverydayEagle|2021-01-11 21:33:03|RT @GrassRootRevo...|            0|      Texas| null|
|       3|        LaRene1|2021-01-11 21:33:22|@akgraham7 @Klapt...|            0|    Florida| null|
|       4|       Crankzie|2021-01-11 21:33:48|@ZitoFN_ LMAO THI...|            0| California| null|
|       5|       senschuh|2021-01-11 21:34:06|RT @jbryan522: Ma...|            0|Mississippi| null|
|       6|RachelleBlidner|2021-01-11 21:34:23|500 people showed...|            0|   New York| null|
|       7|       puffypcb|2021-01-11 21:34:29|@GovRonDeSantis H...|            0|    Florida| null|


In [10]:
df_sp.count()


4698

In [11]:
tweet_df2 = df_sp.select('tweet','location').distinct()

In [12]:
tweet_df2.show()

+--------------------+-------------+
|               tweet|     location|
+--------------------+-------------+
|RT @laggy_mads: Y...|       Oregon|
|I've been trying ...|     New York|
|Pro-life my ass.????|     Colorado|
|RT @ImmunoFever: ...|      Arizona|
|RT @WSJ: U.S. off...|     New York|
|My mom called to ...|     Michigan|
|RT @Mistresshicks...|        Texas|
|RT @lisachristine...|   California|
|@CEUhungary @Elli...| Rhode Island|
|New Jersey reside...|   New Jersey|
|January 21: "Your...|Massachusetts|
|@GovRonDeSantis  ...|     New York|
|RT @JoeNBC: Opera...|  Connecticut|
|RT @drdavidsamadi...|  Mississippi|
|RT @drdavidsamadi...|     Colorado|
|RT @JoeBiden: Fol...|West Virginia|
|RT @MatRyanELATea...|Massachusetts|
|Meanwhile my elde...|     New York|
|RT @okprogressnow...|     Oklahoma|
|RT @thehill: Euro...|     New York|
+--------------------+-------------+
only showing top 20 rows



In [13]:
tweet_df2.count()

4208

In [14]:
tweet_df4 = tweet_df2.select('tweet','location')

In [15]:
pand_df = tweet_df4.toPandas()


In [16]:
pand_df

,tweet,location
0,RT @laggy_mads: Y’all got me fucked up if you ...,Oregon
1,I've been trying to sign my 102-year-old grand...,New York
2,Pro-life my ass.????,Colorado
3,RT @ImmunoFever: Every single vaccination plac...,Arizona
4,RT @WSJ: U.S. officials will suggest anyone ov...,New York
...,...,...
4203,Silver lining of a very dark cloud (Covid) bri...,New Hampshire
4204,"RT @mjcontrera: There are some 70,000 American...",California
4205,RT @DoctorAngry: On @BBCNews I have just seen...,Louisiana
4206,400$ a week for unemployment... as if finding ...,Indiana


In [17]:

host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
# deleting a table in mysql
delete_table="""DROP TABLE IF EXISTS `tweets_locations`;"""
cursor.execute(delete_table)


0

In [18]:

host_name = os.environ.get('host_name')
user = "group2"
passwd = os.environ.get('aws_password')
db="group2db"
# connect to MySQL
con = pymysql.connect(host=host_name,user=user,passwd=passwd,db=db)
cursor = con.cursor()
# creating a table for sending dataframe into mysql
create_table="""CREATE TABLE tweets_locations(tweet_id INT(11) not null auto_increment primary key,  tweet TEXT,
               location VARCHAR(100));"""
cursor.execute(create_table)


0

In [19]:
cols = "`,`".join([str(i) for i in pand_df.columns.tolist()])

# Insert DataFrame records one by one.
for i,row in pand_df.iterrows():
    sql = "INSERT INTO `tweets_locations` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    con.commit()